In [2]:
import numpy as np
import torch 
import os
region=[-25., 25, -30, 30, -10, 60]
N=1000

Seed=5
for seed in range(Seed):
    x01 = np.random.uniform(region[0],region[1],(N, 1))
    x02 = np.random.uniform(region[2],region[3],(N, 1))
    x03 = np.random.uniform(region[4],region[5],(N, 1))
    x0=np.hstack([x01, x02, x03])/10
    x0 = torch.tensor(x0, dtype = torch.float)
    local =  'outputs/LS_28_seed{}/model_best.pkl'.format(seed)
    Net = torch.load(local, map_location='cpu')
    if os.path.exists('pre28_{}.npy'.format(seed)):
        print('presiction has been generated') 
        traj_pre = np.load('pre28_{}.npy'.format(seed))
    else:
        with torch.no_grad():
            traj_pre = Net.predict(x0, h=0.01, steps=20000, returnnp=True)
        pre28 = traj_pre[-10000:].reshape([-1,3])
        np.save('pre28_{}.npy'.format(seed), pre28)
    print(traj_pre.shape)

    local_em =  'outputs/LSem_28_seed{}/model_best.pkl'.format(seed)
    Net_em = torch.load(local_em, map_location='cpu')
    if os.path.exists('pre_em28_{}.npy'.format(seed)):
        print('presiction has been generated') 
        traj_pre_em = np.load('pre_em28_{}.npy'.format(seed))
    else:
        with torch.no_grad():
            traj_pre_em = Net_em.predict(x0, h=0.01, steps=20000, returnnp=True)
        pre_em28 = traj_pre_em[-10000:].reshape([-1,3])
        np.save('pre_em28_{}.npy'.format(seed), pre_em28)
    print(traj_pre_em.shape)


    local_cub =  'outputs/LScub_28_seed{}/model_best.pkl'.format(seed)
    Net_cub = torch.load(local_cub, map_location='cpu')
    if os.path.exists('pre_cub28_{}.npy'.format(seed)):
        print('presiction has been generated') 
        traj_pre_cub = np.load('pre_cub28_{}.npy'.format(seed))
    else:
        with torch.no_grad():
            traj_pre_cub = Net_cub.predict(x0, h=0.01, steps=20000, returnnp=True)
        pre_cub28 = traj_pre_cub[-10000:].reshape([-1,3])
        np.save('pre_cub28_{}.npy'.format(seed), pre_cub28)
    print(traj_pre_cub.shape)


from LSData import LSData
Data=LSData(h=0.01, steps = N, length=1, num_train_traj=0, num_test_traj=0)
if os.path.exists('true28.npy'):
    print('ground truth has been generated') 
    traj_true = np.load('true28.npy')
else:
    traj_true = Data.solver.flow(x0, torch.tensor(0.01), 20000).numpy()
    true28 = traj_true[-10000:].reshape([-1,3])
    np.save('true28', true28)
print(traj_true.shape)

(20001, 1000, 3)
(20001, 1000, 3)
(20001, 1000, 3)
(20001, 1000, 3)
(20001, 1000, 3)
(20001, 1000, 3)
(20001, 1000, 3)
(20001, 1000, 3)
(20001, 1000, 3)
(20001, 1000, 3)
(20001, 1000, 3)
(20001, 1000, 3)
(20001, 1000, 3)
(20001, 1000, 3)
(20001, 1000, 3)
r= 28
(0, 3)
(0, 3)
ground truth has been generated
(20001000, 3)


In [20]:
def den(data):
    ranges = [(-2.5, 2.5), (-3, 3), (-1, 6)]
    bins = 100 
    hist, edges = np.histogramdd(data, bins=bins, range=ranges)
    return hist/data.shape[0]
def error(a):
    return np.sqrt((a**2).mean())

data_true= np.load('true28.npy')
data_pre = np.load('pre28_0.npy')
data_pre_em = np.load('pre_cub28_0.npy')

den_pre=den(data_pre)
den_true=den(data_true)
den_pre_em=den(data_pre_em)

In [23]:
def den(data):
    ranges = [(-2.5, 2.5), (-3, 3), (-1, 6)]
    bins = 100 
    hist, edges = np.histogramdd(data, bins=bins, range=ranges)
    return hist/data.shape[0]
def error(a):
    return np.sqrt((a**2).mean())


data_true= np.load('true28.npy')
den_true=den(data_true)
den_true_norm = error(den_true)


ep=np.zeros([3,Seed])
for seed in range(Seed):
    data_pre = np.load('pre28_{}.npy'.format(seed))
    den_pre=den(data_pre)
    ep[0, seed] = error(den_pre-den_true)/den_true_norm
    
    data_pre_em = np.load('pre_em28_{}.npy'.format(seed))
    den_pre_em=den(data_pre_em)
    ep[1, seed] = error(den_pre_em-den_true)/den_true_norm
    
    data_pre_cub = np.load('pre_cub28_{}.npy'.format(seed))
    den_pre_cub=den(data_pre_cub)
    ep[2, seed] = error(den_pre_cub-den_true)/den_true_norm    
    
print(ep)

[[0.05652964 0.05807825 0.06749816 0.06996137 0.06603222]
 [0.64445686 0.62308852 0.61608897 0.64049005 0.66559997]
 [0.22598963 0.23664187 0.26718506 0.24873702 0.24684482]]


In [24]:
def printvalue(x):
    return format(x, '.2e').replace("e-0", "e-")

print('$e_p$', end=" ")   
for i in range(3):
    print('&', printvalue(ep[i].mean())+'$\pm$'+printvalue(ep[i].std()), end=" ")

$e_p$ & 6.36e-2$\pm$5.33e-3 & 6.38e-1$\pm$1.74e-2 & 2.45e-1$\pm$1.37e-2 